# Connecting to DB in RDS Guide
### Miles Child - November 2022
#### Northeastern Trading Systems

#### Starting DB and importing necessary packages

Once you have created your database in RDS, make sure that it is configured to be publicly accessible and that you've attached a security group that allows inbound/outbound access from 'All Traffic' on both IPv4 and IPv6. Save the login credentials of the DB as soon as you launch the instance.

Next, navigate to your local python environment and install the pymysql package. This process can be very messy if done incorrectly the first time, so make sure you do the following:

For Python 2.x:
- sudo pip install pymysql.

For Python 3.x:
- sudo pip3 install pymysql.

For either running on Anaconda:
- sudo conda install pymysql.

If that didn't work try APT:
- sudo apt-get install pymysql.

credit: https://stackoverflow.com/a/49354387/20355075

Finally, 'import pymysql' in your project before continuing.

In [63]:
# imports
import pymysql
from model.config import *  # credentials stored here

Now, establish a connection to the db by using pymysql.connect(host=<'someurl.rds.amazonaws.com'>, user=<'username'>, password=<'password'>).

Create a cursor, which can essentially be regarded as an object that executes your queries (written locally) on the DB, stores the result in-memory, and returns it back to this project. Check that the connection and cursor have been made properly by checking the version of the db.

In [64]:
connection = pymysql.connect(host=RDS_HOSTNAME, user=RDS_USER, password=RDS_PASSWORD,
 database='nts_lightning_db_v2', # <-- Optional: The name of the database to connect to. Can be left blank and changed later.
  autocommit=True # <-- Optional: If set to True, the connection will autocommit after every query. This is useful for queries that don't modify the database.
  )
cursor = connection.cursor()
cursor.execute('select version()')

1

In [65]:
cursor.execute('select * from strategy')

10

After successfully establishing a connection to the RDS instance, create a DB and add the neccessary tables/entities/attributes.

In [4]:
# Create a new database
cursor.execute("CREATE DATABASE nts_lightning_db;")
cursor.execute(f"CREATE USER 'webapp'@'%' IDENTIFIED BY {DOCKER_DB_PASSWORD};")
cursor.execute("GRANT ALL PRIVILEGES ON nts_lightning_db.* TO 'webapp'@'%';")
cursor.execute("FLUSH PRIVILEGES;")
cursor.execute("USE nts_lightning_db;")

In [ ]:
# Create the tables
cursor.execute("""CREATE TABLE strategy (
    strategy_name varchar(50),
    strategy_id integer AUTO_INCREMENT NOT NULL,
    documentation_link varchar(50),
    launch_date datetime,
    termination_date datetime,
    PRIMARY KEY (strategy_id)
);""")

In [ ]:
cursor.execute("""CREATE TABLE trade (
    trade_id integer UNIQUE NOT NULL,
    strategy_id integer,
    open_time datetime,
    close_time datetime,
    PRIMARY KEY (trade_id),
    CONSTRAINT fk_1 
        FOREIGN KEY (strategy_id) REFERENCES strategy(strategy_id)
        ON UPDATE cascade ON DELETE restrict
);""")

In [ ]:
cursor.execute("""CREATE TABLE trade_leg (
    leg_no integer NOT NULL,
    trade_id integer NOT NULL,
    contract varchar(50),
    open_time datetime,
    close_time datetime,
    PRIMARY KEY(leg_no, trade_id),
    CONSTRAINT fk_2
        FOREIGN KEY (trade_id) REFERENCES trade (trade_id)
        ON UPDATE restrict ON DELETE restrict
);""")

In [ ]:
cursor.execute("""CREATE TABLE fill (
    fill_id integer NOT NULL,
    leg_no integer NOT NULL,
    trade_id integer NOT NULL,
    contract varchar(50),
    qty integer,
    avg float(32),
    placement_time datetime,
    filled_time datetime,
    PRIMARY KEY (fill_id),
    CONSTRAINT fk_3
        FOREIGN KEY (leg_no, trade_id) REFERENCES trade_leg (leg_no, trade_id)
        ON UPDATE cascade ON DELETE restrict
);""")

After creating the database architecture, insert testing (or historical) data into the tables.

note: mockaroo is good for generating dummy data https://www.mockaroo.com/

In [ ]:
# Manual Entry: Dummy data
cursor.execute("""INSERT INTO strategy (strategy_name,strategy_id,documentation_link,launch_date,termination_date) 
values 
('MeanReversionLunchBreak', '1', 'https://github.com/QuantConnect/Lean/blob/master/Algorithm.Python/Alphas/MeanReversionLunchBreakAlpha.py',
 '2022-11-20 00:00:00', NULL);""")


Below is an example of how to insert data programmatically given a list of preexisting SQL insert statements. Websites like https://www.convertcsv.com/csv-to-sql.htm allow you to take a csv and generate SQL insert statements for every entry, which can then be pasted below. 

note: it is also possible to use the build in file reading query in MySQL.

In [7]:
# Fast Entry: Dummy data
# Copy and paste SQL insert statements into the below variable:
txt = """SELECT * FROM strategy""" # <-- Replace with your SQL insert statements
txt = txt.split(';') # <-- Split by the end of each insert statement
for insertion in txt:
    cursor.execute(insertion) 

In [8]:
# Testing that all dummy data was inserted properly and exists in the db.

cursor.execute('SELECT * from fill;')
col_headers = [x[0] for x in cursor.description]
json_data = []
the_data = cursor.fetchall()
for row in the_data:
    json_data.append(dict(zip(col_headers, row)))
json_data

[{'fill_id': 715132,
  'leg_no': 1,
  'trade_id': 51254898,
  'contract': 'TAX',
  'qty': -328,
  'avg': 521.6912281962,
  'placement_time': datetime.datetime(2022, 3, 10, 16, 29, 10),
  'filled_time': datetime.datetime(2022, 3, 10, 16, 29, 17)},
 {'fill_id': 954417,
  'leg_no': 1,
  'trade_id': 452572457,
  'contract': 'CVO',
  'qty': 490,
  'avg': 325.09,
  'placement_time': datetime.datetime(2022, 2, 28, 7, 49, 31),
  'filled_time': datetime.datetime(2022, 2, 28, 7, 49, 41)},
 {'fill_id': 12218623,
  'leg_no': 1,
  'trade_id': 376975473,
  'contract': 'CLLS',
  'qty': -235,
  'avg': 161.3184553646,
  'placement_time': datetime.datetime(2022, 3, 15, 6, 6, 53),
  'filled_time': datetime.datetime(2022, 3, 15, 6, 6, 58)},
 {'fill_id': 14754413,
  'leg_no': 1,
  'trade_id': 44409115,
  'contract': 'WAGE',
  'qty': -407,
  'avg': 447.5675880229,
  'placement_time': datetime.datetime(2022, 10, 1, 5, 5),
  'filled_time': datetime.datetime(2022, 10, 1, 5, 5, 9)},
 {'fill_id': 16642096,
  'le

In [10]:
# Testing get_strategy_info

import pandas as pd

cursor.execute('SELECT * from strategy;')
col_headers = [x[0] for x in cursor.description]
json_data = []
the_data = cursor.fetchall()
for row in the_data:
    json_data.append(dict(zip(col_headers, row)))
df = pd.DataFrame(json_data)
df

,strategy_name,strategy_id,documentation_link,launch_date,termination_date
0,BullBreakout,1,github.com/NTS-Lightning/BullBreakout,2022-09-06 13:28:51,NaT
1,DynamicBreakoutII,2,github.com/NTS-Lightning/DynamicBreakout,2022-07-30 01:32:10,NaT
2,NineEMAVWAPCrossover,3,github.com/NTS-Lightning/NineEMAVWAPCrossover,2022-10-11 22:45:34,NaT
3,ChinaADRArbitrage,4,github.com/NTS-Lightning/ChinaADRArbitrage,2022-10-02 05:00:36,NaT
4,LongFTX,5,github.com/NTS-Lightning/LongFTX,2022-06-01 04:08:38,NaT
5,MeanReversionLunchbreak,6,github.com/NTS-Lightning/MeanReversion,2022-09-07 05:22:14,NaT
6,ModifiedVWAPReversion,7,github.com/NTS-Lightning/ModifiedVWAPReversion,2022-06-08 23:42:50,NaT
7,OrnamentalGourdFuturesArbitrage,8,github.com/NTS-Lightning/OrnamentalGourdFuture...,2022-02-21 15:50:43,2022-12-03
8,BuyHighSellLowOnMargin,9,github.com/NTS-Lightning/BuyHighSellLowOnMargin,2022-10-11 14:35:16,NaT
9,GOHMRebasePoolArbitrage,10,github.com/NTS-Lightning/GOHMRebasePoolArbitrage,2022-02-13 09:19:34,NaT


In [ ]:
# Testing get_trade_info

# TODO: FIX QUERY

cursor.execute("""
SELECT * from trade join trade_leg on trade.trade_id = trade_leg.trade_id join fill on trade_leg.leg_no = fill.leg_no and trade_leg.trade_id = fill.trade_id
where trade.trade_id = '000000002'
order by trade_leg.leg_no, fill.placement_time;
""")
col_headers = [x[0] for x in cursor.description]
json_data = []
the_data = cursor.fetchall()
for row in the_data:
    json_data.append(dict(zip(col_headers, row)))
df = pd.DataFrame(json_data)
df


In [ ]:
# Testing get_trade_leg_info
cursor.execute("""
Select * from trade_leg join fill on trade_leg.leg_no = fill.leg_no and trade_leg.trade_id = fill.trade_id
where trade_leg.trade_id = '000000002' and trade_leg.leg_no = '1'
order by fill.placement_time;
""")
col_headers = [x[0] for x in cursor.description]
json_data = []
the_data = cursor.fetchall()
for row in the_data:
    json_data.append(dict(zip(col_headers, row)))
df = pd.DataFrame(json_data)
df

In [ ]:
# Testing get_fill_info
cursor.execute("""
Select * from fill where fill.fill_id = '000000002';
""")
col_headers = [x[0] for x in cursor.description]
json_data = []
the_data = cursor.fetchall()
for row in the_data:
    json_data.append(dict(zip(col_headers, row)))
json_data
#df = pd.DataFrame(json_data)
#df

In [60]:
import pandas as pd

# Testing get_historical_trades
strategy = 'MeanReversionLunchBreak'
lookback = '2022-06-01 00:00:00'
cursor.execute(f"""
    Select trade.trade_id as trade_id, trade.open_time as open_time, trade.close_time as close_time, trade_leg.contract as contract,
    max(trade_leg.leg_no) as no_legs, count(fill.fill_id) as fills, sum(fill.qty * fill.avg) * -1 as pnl 
    from strategy 
    join trade on strategy.strategy_id = trade.strategy_id 
    join trade_leg on trade.trade_id = trade_leg.trade_id 
    join fill on trade_leg.leg_no = fill.leg_no and trade_leg.trade_id = fill.trade_id
    where strategy.strategy_name = '{strategy}'
    group by trade_id
    order by trade.open_time;
    """)
col_headers = [x[0] for x in cursor.description]
json_data = []
the_data = cursor.fetchall()
for row in the_data:
    json_data.append(dict(zip(col_headers, row)))
df = pd.DataFrame(json_data)
df.head(10)

,trade_id,open_time,close_time,contract,no_legs,fills,pnl
0,23767170,2022-02-12 01:08:57,2022-02-12 06:12:57,CRBP,1,2,879.273623
1,708353551,2022-02-16 22:16:52,2022-02-17 06:22:52,ATHM,1,2,46139.436585
2,26853316,2022-02-28 06:21:50,2022-02-28 11:24:50,VSH,1,2,43736.596687
3,244628470,2022-04-01 04:55:36,2022-04-01 09:19:36,KMT,1,2,7074.267729
4,230039270,2022-04-12 07:51:07,2022-04-12 15:08:07,DSWL,1,2,-5046.174471
5,798894307,2022-04-26 11:50:18,2022-04-26 17:51:18,EMES,1,2,8952.132478
6,381145924,2022-04-29 13:30:25,2022-04-29 16:27:25,NEWM,1,2,39493.262038
7,300585899,2022-04-30 12:48:47,2022-04-30 15:17:47,COF^D,1,2,3897.496265
8,668865077,2022-05-01 07:17:29,2022-05-01 07:43:29,HTHT,1,2,68.540729
9,504316197,2022-05-01 08:30:35,2022-05-01 16:39:35,FPAY,1,2,75808.560845


In [57]:
# Testing get_open_trades
strategy = 'LongFTX'
cursor.execute(f"""
    Select trade.trade_id as trade_id, trade.open_time as open_time, trade_leg.contract as contract,
    max(trade_leg.leg_no) as no_legs, count(fill.fill_id) as fills, sum(fill.qty * fill.avg) as capital_out
    from strategy 
    join trade on strategy.strategy_id = trade.strategy_id 
    join trade_leg on trade.trade_id = trade_leg.trade_id 
    join fill on trade_leg.leg_no = fill.leg_no and trade_leg.trade_id = fill.trade_id
    where strategy.strategy_name = '{strategy}' and trade.close_time is NULL
    group by trade_id
    order by trade.open_time;
    """)
col_headers = [x[0] for x in cursor.description]
json_data = []
the_data = cursor.fetchall()
for row in the_data:
    json_data.append(dict(zip(col_headers, row)))
df = pd.DataFrame(json_data)
df.head(10)

,trade_id,open_time,contract,no_legs,fills,capital_out
0,614004145,2022-02-02 18:12:15,SPE^B,1,2,-37715.416504
1,608530551,2022-02-11 02:47:01,GAB,1,2,15216.026030
2,517518405,2022-02-27 00:10:18,BMO,1,2,17670.764385
3,23775564,2022-03-09 14:34:59,PEIX,1,2,-6355.704431
4,376975473,2022-03-15 05:35:58,CLLS,1,2,-7275.237011
5,539056113,2022-04-09 05:54:46,B,1,2,-3337.252564
6,171327069,2022-04-21 10:28:35,TPVZ,1,2,-46534.866062
7,550413487,2022-04-29 18:38:24,WAFDW,1,2,-9336.061048
8,868094915,2022-05-10 07:31:14,AKBA,1,2,-38488.508045
9,942431792,2022-05-30 11:14:42,TWNKW,1,2,-2411.573140


In [11]:
cursor.execute('select * from strategy where termination_date is null')
col_headers = [x[0] for x in cursor.description]
json_data = []
the_data = cursor.fetchall()
for row in the_data:
    json_data.append(dict(zip(col_headers, row)))

# remove all of the documentation_link and termination_date fields from the json_data
for i in range(len(json_data)):
    del json_data[i]['documentation_link']
    del json_data[i]['termination_date']
json_data

[{'strategy_name': 'BullBreakout',
  'strategy_id': 1,
  'launch_date': datetime.datetime(2022, 9, 6, 13, 28, 51)},
 {'strategy_name': 'DynamicBreakoutII',
  'strategy_id': 2,
  'launch_date': datetime.datetime(2022, 7, 30, 1, 32, 10)},
 {'strategy_name': 'NineEMAVWAPCrossover',
  'strategy_id': 3,
  'launch_date': datetime.datetime(2022, 10, 11, 22, 45, 34)},
 {'strategy_name': 'ChinaADRArbitrage',
  'strategy_id': 4,
  'launch_date': datetime.datetime(2022, 10, 2, 5, 0, 36)},
 {'strategy_name': 'LongFTX',
  'strategy_id': 5,
  'launch_date': datetime.datetime(2022, 6, 1, 4, 8, 38)},
 {'strategy_name': 'MeanReversionLunchbreak',
  'strategy_id': 6,
  'launch_date': datetime.datetime(2022, 9, 7, 5, 22, 14)},
 {'strategy_name': 'ModifiedVWAPReversion',
  'strategy_id': 7,
  'launch_date': datetime.datetime(2022, 6, 8, 23, 42, 50)},
 {'strategy_name': 'BuyHighSellLowOnMargin',
  'strategy_id': 9,
  'launch_date': datetime.datetime(2022, 10, 11, 14, 35, 16)},
 {'strategy_name': 'GOHMRebas

In [55]:
import pandas as pd
# Get the daily pnl across all trades for all data
cursor.execute("""
    Select trade.open_time as Date, sum(fill.qty * fill.avg) * -1 as pnl
    from trade
    join trade_leg on trade.trade_id = trade_leg.trade_id
    join fill on trade_leg.leg_no = fill.leg_no and trade_leg.trade_id = fill.trade_id
    group by trade.open_time
    order by trade.open_time;
    """)

col_headers = [x[0] for x in cursor.description]
json_data = []
the_data = cursor.fetchall()
for row in the_data:
    json_data.append(dict(zip(col_headers, row)))

df = pd.DataFrame(json_data)
# Group the open_time by day
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')
df = df.groupby(pd.Grouper(freq='D')).sum()
df = df.reset_index()
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

# Convert the dataframe to a JSON object, preserving the indeces
col_headers = [x for x in df.columns]
json_data = []
for entry in df.values:
    json_data.append(dict(zip(col_headers, entry)))
json_data


[{'Date': '2022-01-01', 'pnl': -16415.985167384002},
 {'Date': '2022-01-02', 'pnl': 0.0},
 {'Date': '2022-01-03', 'pnl': -11154.757608817003},
 {'Date': '2022-01-04', 'pnl': 2542.4355028624996},
 {'Date': '2022-01-05', 'pnl': -54291.1398328314},
 {'Date': '2022-01-06', 'pnl': -21504.503576147195},
 {'Date': '2022-01-07', 'pnl': 0.0},
 {'Date': '2022-01-08', 'pnl': 0.0},
 {'Date': '2022-01-09', 'pnl': 0.0},
 {'Date': '2022-01-10', 'pnl': 0.0},
 {'Date': '2022-01-11', 'pnl': 38554.45584222958},
 {'Date': '2022-01-12', 'pnl': 0.0},
 {'Date': '2022-01-13', 'pnl': 28637.110495943},
 {'Date': '2022-01-14', 'pnl': 2216.773297520802},
 {'Date': '2022-01-15', 'pnl': 0.0},
 {'Date': '2022-01-16', 'pnl': 0.0},
 {'Date': '2022-01-17', 'pnl': 0.0},
 {'Date': '2022-01-18', 'pnl': 0.0},
 {'Date': '2022-01-19', 'pnl': -47000.929231612},
 {'Date': '2022-01-20', 'pnl': -24412.011792547506},
 {'Date': '2022-01-21', 'pnl': 4124.1795432117},
 {'Date': '2022-01-22', 'pnl': -11736.844127702701},
 {'Date': '2

In [50]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

lookback = 6
lookback = datetime.now() - relativedelta(months=lookback)
lookback = lookback.strftime("%Y-%m-%d %H:%M:%S")
lookback

'2022-06-13 01:07:03'

In [62]:
# Fixing daily pnl
strategy = 'LongFTX'
cursor.execute(f"""
                Select trade.open_time as Date, sum(fill.qty * fill.avg) * -1 as PNL
                from trade
                join trade_leg on trade.trade_id = trade_leg.trade_id
                join fill on trade_leg.leg_no = fill.leg_no and trade_leg.trade_id = fill.trade_id
                join strategy on trade.strategy_id = strategy.strategy_id
                where strategy.strategy_name = "{strategy}"
                group by trade.open_time
                order by trade.open_time;
                """)
# Convert this output to a dataframe
col_headers = [x[0] for x in cursor.description]
json_data = []
the_data = cursor.fetchall()
for row in the_data:
    json_data.append(dict(zip(col_headers, row)))
df = pd.DataFrame(json_data)

,Date,PNL
0,2022-02-02 18:12:15,37715.416504
1,2022-02-11 02:47:01,-15216.026030
2,2022-02-27 00:10:18,-17670.764385
3,2022-03-09 14:34:59,6355.704431
4,2022-03-15 05:35:58,7275.237011
5,2022-04-09 05:54:46,3337.252564
6,2022-04-21 10:28:35,46534.866062
7,2022-04-29 18:38:24,9336.061048
8,2022-05-10 07:31:14,38488.508045
9,2022-05-30 11:14:42,2411.573140
